# sqlalchemy

Recommended exercises: <https://www.w3resource.com/sql-exercises/>

Another place to just practice SQL queries: <http://sqlfiddle.com>

sqlalchemy allows us to connect and interact with databases from within Python

Most of your SQL experience will be with connecting to a database that already exists. Most data analysts / data scientists are not the data base administrator and this is not a database administration course.

You can download the chinook database file from the chinook database github:

https://github.com/lerocha/chinook-database/tree/master/ChinookDatabase/DataSources

In [1]:
import pandas as pd

In [2]:
from sqlalchemy import create_engine

# create_engine creates a connection to an existing database
# I have 'Chinook_sqlite.sqlite' downloaded into my folder, and python
# connects to this database
engine = create_engine('sqlite:///Chinook_Sqlite.sqlite')

In [3]:
# Save the table names to a list: table_names
table_names = engine.table_names()

# Print the table names to the shell
print(table_names)

['Album', 'Artist', 'Customer', 'Employee', 'Genre', 'Invoice', 'InvoiceLine', 'MediaType', 'Playlist', 'PlaylistTrack', 'Track']


Once we have the database engine created, we can begin executing SQL queries by establishing a connection with the database.

In [6]:
# Open engine connection
con = engine.connect()

# Perform query
rs = con.execute('SELECT * FROM Album')

# Save results of the query to DataFrame
df = pd.DataFrame(rs.fetchall())

# Close connection
con.close()

# Print head of query results
print(df.head())
print(rs.keys())

   0                                      1  2
0  1  For Those About To Rock We Salute You  1
1  2                      Balls to the Wall  2
2  3                      Restless and Wild  2
3  4                      Let There Be Rock  1
4  5                               Big Ones  3
['AlbumId', 'Title', 'ArtistId']


Instead of having to open and close the engine connection, we can use Python's with statement which will automatically open and close the connection for us

In [7]:
# We can write our SQL command enclosed in triple quotes
command = '''
SELECT LastName, Title 
FROM Employee
'''

# SELECT chooses the desired columns
# FROM indicates the table to query

with engine.connect() as con:
    rs = con.execute(command)
    df = pd.DataFrame(rs.fetchall())
    df.columns = rs.keys()

print(df)

   LastName                Title
0     Adams      General Manager
1   Edwards        Sales Manager
2   Peacock  Sales Support Agent
3      Park  Sales Support Agent
4   Johnson  Sales Support Agent
5  Mitchell           IT Manager
6      King             IT Staff
7  Callahan             IT Staff


Pandas offers functionality to directly query a SQL database using an existing engine

In [8]:
# we can use the same command as earlier:
df = pd.read_sql_query(command, engine)
df

,LastName,Title
0,Adams,General Manager
1,Edwards,Sales Manager
2,Peacock,Sales Support Agent
3,Park,Sales Support Agent
4,Johnson,Sales Support Agent
5,Mitchell,IT Manager
6,King,IT Staff
7,Callahan,IT Staff


ORDER BY is SQL's version of sort

<https://www.w3schools.com/sql/sql_orderby.asp>

`
SELECT column1, column2, ...
FROM table_name
ORDER BY column1, column2, ... ASC|DESC;
`

In [11]:
command = '''
SELECT * 
FROM Employee 
ORDER BY Birthdate DESC
'''
pd.read_sql_query(command, engine)

,EmployeeId,LastName,FirstName,Title,ReportsTo,BirthDate,HireDate,Address,City,State,Country,PostalCode,Phone,Fax,Email
0,3,Peacock,Jane,Sales Support Agent,2.0,1973-08-29 00:00:00,2002-04-01 00:00:00,1111 6 Ave SW,Calgary,AB,Canada,T2P 5M5,+1 (403) 262-3443,+1 (403) 262-6712,jane@chinookcorp.com
1,6,Mitchell,Michael,IT Manager,1.0,1973-07-01 00:00:00,2003-10-17 00:00:00,5827 Bowness Road NW,Calgary,AB,Canada,T3B 0C5,+1 (403) 246-9887,+1 (403) 246-9899,michael@chinookcorp.com
2,7,King,Robert,IT Staff,6.0,1970-05-29 00:00:00,2004-01-02 00:00:00,590 Columbia Boulevard West,Lethbridge,AB,Canada,T1K 5N8,+1 (403) 456-9986,+1 (403) 456-8485,robert@chinookcorp.com
3,8,Callahan,Laura,IT Staff,6.0,1968-01-09 00:00:00,2004-03-04 00:00:00,923 7 ST NW,Lethbridge,AB,Canada,T1H 1Y8,+1 (403) 467-3351,+1 (403) 467-8772,laura@chinookcorp.com
4,5,Johnson,Steve,Sales Support Agent,2.0,1965-03-03 00:00:00,2003-10-17 00:00:00,7727B 41 Ave,Calgary,AB,Canada,T3B 1Y7,1 (780) 836-9987,1 (780) 836-9543,steve@chinookcorp.com
5,1,Adams,Andrew,General Manager,NaN,1962-02-18 00:00:00,2002-08-14 00:00:00,11120 Jasper Ave NW,Edmonton,AB,Canada,T5K 2N1,+1 (780) 428-9482,+1 (780) 428-3457,andrew@chinookcorp.com
6,2,Edwards,Nancy,Sales Manager,1.0,1958-12-08 00:00:00,2002-05-01 00:00:00,825 8 Ave SW,Calgary,AB,Canada,T2P 2T3,+1 (403) 262-3443,+1 (403) 262-3322,nancy@chinookcorp.com
7,4,Park,Margaret,Sales Support Agent,2.0,1947-09-19 00:00:00,2003-05-03 00:00:00,683 10 Street SW,Calgary,AB,Canada,T2P 5G3,+1 (403) 263-4423,+1 (403) 263-4289,margaret@chinookcorp.com


Filter row selection with WHERE. (similar to using if as a boolean mask)

SQL uses single equal sign = for comparison

In [13]:
command = '''
SELECT * 
FROM Employee 
WHERE EmployeeId >= 6 AND Title = 'IT Staff'
ORDER BY BirthDate
'''
pd.read_sql_query(command, engine)

,EmployeeId,LastName,FirstName,Title,ReportsTo,BirthDate,HireDate,Address,City,State,Country,PostalCode,Phone,Fax,Email
0,8,Callahan,Laura,IT Staff,6,1968-01-09 00:00:00,2004-03-04 00:00:00,923 7 ST NW,Lethbridge,AB,Canada,T1H 1Y8,+1 (403) 467-3351,+1 (403) 467-8772,laura@chinookcorp.com
1,7,King,Robert,IT Staff,6,1970-05-29 00:00:00,2004-01-02 00:00:00,590 Columbia Boulevard West,Lethbridge,AB,Canada,T1K 5N8,+1 (403) 456-9986,+1 (403) 456-8485,robert@chinookcorp.com


We can look at data across multiple tables using a JOIN

LIMIT acts like "head()", and limits the number of entries it returns

In [20]:
command = '''
SELECT * 
FROM Album
LIMIT 10
'''
pd.read_sql_query(command, engine)

,AlbumId,Title,ArtistId
0,1,For Those About To Rock We Salute You,1
1,2,Balls to the Wall,2
2,3,Restless and Wild,2
3,4,Let There Be Rock,1
4,5,Big Ones,3
5,6,Jagged Little Pill,4
6,7,Facelift,5
7,8,Warner 25 Anos,6
8,9,Plays Metallica By Four Cellos,7
9,10,Audioslave,8


In [21]:
command = '''
SELECT * 
FROM Album
INNER JOIN Artist ON Album.ArtistId = Artist.ArtistId
LIMIT 10
'''
pd.read_sql_query(command, engine)

,AlbumId,Title,ArtistId,ArtistId,Name
0,1,For Those About To Rock We Salute You,1,1,AC/DC
1,2,Balls to the Wall,2,2,Accept
2,3,Restless and Wild,2,2,Accept
3,4,Let There Be Rock,1,1,AC/DC
4,5,Big Ones,3,3,Aerosmith
5,6,Jagged Little Pill,4,4,Alanis Morissette
6,7,Facelift,5,5,Alice In Chains
7,8,Warner 25 Anos,6,6,Antônio Carlos Jobim
8,9,Plays Metallica By Four Cellos,7,7,Apocalyptica
9,10,Audioslave,8,8,Audioslave


In [30]:
command = '''
SELECT Artist.ArtistId, Name, COUNT(AlbumId) AS album_count
FROM Album
INNER JOIN Artist ON Album.ArtistId = Artist.ArtistId
GROUP BY Artist.ArtistId
LIMIT 10
'''
pd.read_sql_query(command, engine)

,ArtistId,Name,album_count
0,1,AC/DC,2
1,2,Accept,2
2,3,Aerosmith,1
3,4,Alanis Morissette,1
4,5,Alice In Chains,1
5,6,Antônio Carlos Jobim,2
6,7,Apocalyptica,1
7,8,Audioslave,3
8,9,BackBeat,1
9,10,Billy Cobham,1


In [32]:
command = '''
SELECT * 
FROM Album
ORDER BY ArtistId
LIMIT 15
'''

pd.read_sql_query(command, engine)

,AlbumId,Title,ArtistId
0,1,For Those About To Rock We Salute You,1
1,4,Let There Be Rock,1
2,2,Balls to the Wall,2
3,3,Restless and Wild,2
4,5,Big Ones,3
5,6,Jagged Little Pill,4
6,7,Facelift,5
7,8,Warner 25 Anos,6
8,34,Chill: Brazil (Disc 2),6
9,9,Plays Metallica By Four Cellos,7


In [33]:
# Conditionals on the Group By must be done with 'HAVING'
command = '''
SELECT Artist.ArtistId, Name, COUNT(AlbumId) AS album_count
FROM Album
INNER JOIN Artist ON Album.ArtistId = Artist.ArtistId
GROUP BY Artist.ArtistId
HAVING album_count > 8
'''
pd.read_sql_query(command, engine)

,ArtistId,Name,album_count
0,22,Led Zeppelin,14
1,50,Metallica,10
2,58,Deep Purple,11
3,90,Iron Maiden,21
4,150,U2,10
